In [13]:
import pandas as pd
import numpy as np

In [ ]:
merged_df_path = '../dataset/join_result.csv'

## Perform various calculations on the merged dataset

In [15]:
#List of interesting metrics
metric_list = ['stars','forks','commits','pull_requests','contributors']

#Accumulate metric value between each month
def accumulate_metrics(df,metric_of_interest, amount_of_months, output=''):
    for j in df['url']:
        cumulative_value = df.loc[df['url'] == j,metric_of_interest + '_at_submission']
        for i in range(1, amount_of_months+1):
            cumulative_value += df[metric_of_interest + '_month_' +str(i)]
            df.loc[df['url'] == j,metric_of_interest + '_month_' +str(i)] = cumulative_value
    
    if output != '':
        df.to_csv(output, index=False)

#Accumulate all metric value between each month except contributors
def accumulate_all_metrics(df, amount_of_months, output_path):
    for metric in metric_list:
        if metric == 'contributors':
            continue
        accumulate_metrics(df,metric,amount_of_months,output_path)


#Calculate distance of a metric between 2 time periods
def calculate_dist(df, value1, value2):
    return df[value2] - df[value1]

#Create a column for distance of metric
def create_dist(df, value1, value2, result_name):
    df[result_name] = calculate_dist(df, value1, value2)

#Calculate percent change of a metric between 2 time periods
def calculate_percentage(df,value1,value2):
    return ((df[value2]/df[value1]) * 100) - 100 

#Create a column for percent change of metric
def create_percentage(df,value1,value2, result_name):
    percent_result = calculate_percentage(df,value1,value2)
    #Replace inf value with 0
    percent_result.replace([np.inf, -np.inf], np.nan, inplace=True)
    percent_result.fillna(0, inplace=True)
    df[result_name] = percent_result.apply(
        lambda x: 0 if x == np.nan else x
    )

#Run all function above for all metrics between each months
def create_dist_percent(df,amount_of_month,output=''):
    for metric in metric_list:
        create_dist(df, metric+'_at_submission', metric+'_month_1', 'dist_'+metric+'(submission-month_1)')
        create_percentage(df, metric+'_at_submission', metric+'_month_1', 'percent_'+metric+'(submission-month_1)')
    
        for i in range(1,amount_of_month):
            create_dist(df, metric+'_month_' + str(i), metric+'_month_' + str(i+1), 'dist_'+metric+'(month_' + str(i) + '-month_'+str(i+1)+')')
            create_percentage(df, metric+'_month_' + str(i), metric+'_month_' + str(i+1), 'percent_'+metric+'(month_' + str(i) + '-month_'+str(i+1)+')')

    if output != '':
        df.to_csv(output, index=False)


In [16]:
join_df = pd.read_csv(merged_df_path)

accumulate_all_metrics(join_df,5,merged_df_path)
create_dist_percent(join_df,5,merged_df_path)